<a href="https://colab.research.google.com/github/YuminosukeSato/minddrone/blob/main/CNNmodelpreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import math
from sklearn.manifold import TSNE
from matplotlib import pyplot
import pickle
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
import tqdm
from torch.autograd import Variable
def preprocessing(floatArray):
    window = np.hamming(125)
    w = np.empty(125)
    d2 = np.empty((floatArray.shape[0],27*16))
    for i in range(floatArray.shape[0]-125):
        for j in range(15):
            w = np.abs(np.fft.fftn(floatArray[i:i+125,j]*window))
            d2[i,27*j:27*j+26] = np.log10(1 + w[4:30])
    return d2
def labeler(array):
  label = np.empty(175)
  for i in range(175):
    if i <= 25:
      label[i] = 0 #0 is normal
    elif i <= 50:
      label[i] = 1 #1is forward
    elif i <= 75:
      label[i] = 2 # 2 is righet
    elif i <= 100:
      label[i] = 3
    elif i <= 125:
      label[i] = 4
    elif i <= 150:
      label[i] = 5
    else:
      label[i] = 6
  return label


if __name__ == "__main__":
    data = np.loadtxt("/content/OpenBCI-RAW-2022-02-16_16-33-39.txt",dtype='str',delimiter=",",skiprows=5)
    b = data[:,1:17]
    floatArray = b.astype(float)
    d2 = np.empty((floatArray.shape[0],27*16))
    d2 = preprocessing(floatArray)
    ts = d2[125*60-1:125*130-1,:]
    ts=torch.Tensor(ts)
    ts = ts.view(175,50,432)
    label = labeler(ts)
    train_data, test_data, train_label, test_label = train_test_split(ts, label, test_size=0.2,shuffle = False)
    train_x = torch.Tensor(train_data)
    test_x = torch.Tensor(test_data)
    #train_y = torch.LongTensor(train_label)  # torch.int64のデータ型に
    #test_y = torch.LongTensor(test_label)
    train_y = torch.Tensor(train_label)
    test_y = torch.Tensor(test_label)
    train_dataset = TensorDataset(train_x, train_y)
    test_dataset = TensorDataset(test_x, test_y)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(test_dataset,batch_size=128,shuffle=True)

In [12]:
import torch
import torch.nn as nn
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 60,kernel_size=(1, 15),stride=(1,3))
        self.conv2 = nn.Conv2d(60, 60, kernel_size=(1, 4),stride=(1,2))
        self.conv3 = nn.Conv2d(60, 60, kernel_size=(1, 17),stride=(1,3))
        self.conv4 = nn.Conv2d(60, 90, kernel_size=(1, 3),stride=(1,1))
        self.conv5 = nn.Conv2d(90, 120, kernel_size=(1, 1),stride=(1,1))
        self.pool = nn.MaxPool2d(kernel_size=(1, 2),stride=(1,2))
        self.soft = nn.Softmax()
        self.li=nn.Linear(7,184320)

    def forward(self, x):
      x = self.conv1(x)
      x = self.pool(x)
      x = self.conv2(x)
      x = self.pool(x)
      x = self.conv3(x)
      x = self.conv4(x)
      x = self.pool(x)
      x = self.conv5(x)
      #x = self.li(x)
      #x = self.soft(x)
      return x

In [16]:
net = Net()
x = torch.ones(128,1,21,432)
o = net(x)
print(o)

RuntimeError: ignored

In [10]:
net = Net()

# loss関数の定義
criterion = nn.CrossEntropyLoss()

# 最適化関数の定義
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
### トレーニング ###
for epoch in range(100):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # データ[input, labels]を取得
        inputs, labels = data
        print(inputs.shape)
        # 均配の初期化
        optimizer.zero_grad()
        inputs = inputs.view(128,1,50,432)
        print(inputs.shape)
        #フォワード処理forward関数の実行）
        outputs = net(inputs)
        # lossを計算
        loss = criterion(outputs, labels)
        print(loss)
        # lossから均配を計算
        loss.backward()

        # 最適化(均配からパラメータの更新)
        optimizer.step()

        # 学習過程を表示
        running_loss += loss.item()
        if i % 2000 == 1999: # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % 
                    (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

# 次回から学習しなくていいように、学習済みモデルのパラメータを"net_00.prmとして保存"
params = net.state_dict()
torch.save(params, "net_00.prm", pickle_protocol=4)

print('Finished Training')

### 学習済みモデルのテスト ###
correct = 0
total = 0

with torch.no_grad(): # パラメータ保持の停止（test時には必要ないので。）
    for data in val_loader:
        signal, labels = data
        signal = torch.unsqueeze(signal,1)
        signal = torch.unsqueeze(signal,1)

        # 各ラベルの確率がoutputに出力される
        outputs = net(signal)

        # torch.maxは最大値とそのインデックス(ラベル)を返す
        # 最大値は無視してラベルのみを保存
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 正解率を出力
print('Accuracy of the network on the 10000 test images: %d %%' %
        (100 * correct / total))

torch.Size([128, 50, 432])
torch.Size([128, 1, 50, 432])


RuntimeError: ignored

In [ ]:
correct = 0
total = 0

with torch.no_grad(): # パラメータ保持の停止（test時には必要ないので。）
    for data in val_loader:
        signal, labels = data
        signal = torch.unsqueeze(signal,1)
        signal = torch.unsqueeze(signal,1)
        # 各ラベルの確率がoutputに出力される
        outputs = net(signal)

        # torch.maxは最大値とそのインデックス(ラベル)を返す
        # 最大値は無視してラベルのみを保存
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# 正解率を出力
print('Accuracy of the network on the 10000 test images: %d %%' %
        (100 * correct / total))


tensor([[[109, 109, 109, 109, 109,  73]],

        [[109, 109,  73, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[ 73,  73, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  79]],

        [[ 73, 109, 109,  40, 109,  73]],

        [[ 73, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[ 73, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  79]],

        [[ 52, 109,  73,  52,  73,  89]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[109, 109, 109, 109, 109,  73]],

        [[1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: ignored